In [1]:
import os
import time
import numpy as np
import pandas as pd
import tflite_runtime.interpreter as tflite
import gc

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, precision_score, recall_score

/home/pi/myenv/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/pi/myenv/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/pi/myenv/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/pi/myenv/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# ---------------------------
# 1. Load the Exported Test Set
# ---------------------------

# Set the path to the exported test set CSV
TEST_CSV = './exported_test_set.csv'

if not os.path.exists(TEST_CSV):
    raise FileNotFoundError(f"Test set CSV not found at {TEST_CSV}")

# Load the test set from CSV
X_test_export = pd.read_csv(TEST_CSV)

# Assume the target column is named "target"
Y_test = X_test_export["target"].values
X_test = X_test_export.drop(columns=["target"])

print(f"Test set loaded from {TEST_CSV}")
print(f"X_test shape: {X_test.shape}")

Test set loaded from ./exported_test_set.csv
X_test shape: (41992, 83)


In [3]:
# ---------------------------
# 2. Define the Preprocessor
# ---------------------------
# Identify columns (you must know these from training)
numerical_cols = X_test.select_dtypes(include=["int64", "float64", "float32"]).columns.tolist()
categorical_cols = ["proto", "service"]

# Create new transformers
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Fit on test set (or a subset) and transform test data
X_test_transformed = preprocessor.fit_transform(X_test)
print(f"X_test_transformed shape: {X_test_transformed.shape}")

X_test_transformed shape: (41992, 94)


In [4]:
# ---------------------------
# 2. Load TFLite Float32 Model and Prepare for Inference
# ---------------------------
tflite_model_path = "./tflite_models/best_AE_QAT_model.tflite"  # update this path as needed
if not os.path.exists(tflite_model_path):
    raise FileNotFoundError(f"TFLite model not found at {tflite_model_path}")

interpreter = tflite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Display Input Details
print("Input Details with Quantization Parameters:")
for input_detail in input_details:
    print(f"  Name: {input_detail['name']}")
    print(f"  Shape: {input_detail['shape']}")
    print(f"  Data Type: {input_detail['dtype']}")
    print(f"  Quantization Parameters: {input_detail['quantization']}")
    print(f"  Quantization Scale: {input_detail['quantization_parameters']['scales']}")
    print(f"  Quantization Zero Points: {input_detail['quantization_parameters']['zero_points']}\n")

# Display Output Details
print("\nOutput Details with Quantization Parameters:")
for idx, output_detail in enumerate(output_details):
    print(f"Output {idx}:")
    print(f"  Name: {output_detail['name']}")
    print(f"  Shape: {output_detail['shape']}")
    print(f"  Data Type: {output_detail['dtype']}")
    print(f"  Quantization Parameters: {output_detail['quantization']}")
    print(f"  Quantization Scale: {output_detail['quantization_parameters']['scales']}")
    print(f"  Quantization Zero Points: {output_detail['quantization_parameters']['zero_points']}\n")


Input Details with Quantization Parameters:
  Name: serving_default_encoder_input:0
  Shape: [ 1 94]
  Data Type: <class 'numpy.int8'>
  Quantization Parameters: (1.2719604969024658, -125)
  Quantization Scale: [1.2719605]
  Quantization Zero Points: [-125]


Output Details with Quantization Parameters:
Output 0:
  Name: StatefulPartitionedCall:1
  Shape: [ 1 94]
  Data Type: <class 'numpy.int8'>
  Quantization Parameters: (0.09761205315589905, -111)
  Quantization Scale: [0.09761205]
  Quantization Zero Points: [-111]

Output 1:
  Name: StatefulPartitionedCall:0
  Shape: [ 1 13]
  Data Type: <class 'numpy.int8'>
  Quantization Parameters: (0.00390625, -128)
  Quantization Scale: [0.00390625]
  Quantization Zero Points: [-128]



In [5]:
# Retrieve quantization parameters
input_scale, input_zero_point = input_details[0]['quantization']
output_scale, output_zero_point = output_details[1]['quantization']

print(f"Input quantization: scale = {input_scale}, zero_point = {input_zero_point}")
print(f"Output quantization: scale = {output_scale}, zero_point = {output_zero_point}")

Input quantization: scale = 1.2719604969024658, zero_point = -125
Output quantization: scale = 0.00390625, zero_point = -128


In [6]:
# ---------------------------
# Define Helper Functions for Quantization
# ---------------------------
def quantize_input(data, scale, zero_point):
    """
    Quantizes a float32 input array to int8 using the provided scale and zero point.
    """
    return np.round(data / scale + zero_point).astype(np.int8)

def dequantize_output(data_int8, scale, zero_point):
    """
    Converts quantized int8 output back to float32.
    """
    return (data_int8.astype(np.float32) - zero_point) * scale

def run_inference_tflite(input_data):
    """
    Run inference on the TFLite model using pre-quantized input data and return the raw output.
    
    This function assumes that `input_data` is already pre-quantized (if the model is quantized)
    and does not perform any dequantization on the output.
    """
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

In [7]:
# ---------------------------
# 3. Single Sample Inference and Metrics (Int8 Quantized for AE)
# ---------------------------
X_test_processed = X_test_transformed.astype(np.float32)

# Choose a sample index (modify as needed)
sample_index = 5  # for example, first sample
sample = X_test_processed[sample_index:sample_index+1]

# Quantize the sample using the model's input quantization parameters.
# The quantize_input function is assumed to be defined as:
# def quantize_input(data, scale, zero_point):
#     return np.round(data / scale + zero_point).astype(np.int8)
sample_quantized = quantize_input(sample, input_scale, input_zero_point)

# Run inference: set the input tensor with the pre-quantized sample and invoke.
interpreter.set_tensor(input_details[0]['index'], sample_quantized)
interpreter.invoke()

# Use output index 1 for classification predictions.
print("Using output index 1 for classification predictions.")
predicted_output = interpreter.get_tensor(output_details[1]['index'])

# Dequantize the output using the provided dequantize_output function:
# def dequantize_output(data_int8, scale, zero_point):
#     return (data_int8.astype(np.float32) - zero_point) * scale
output_scale, output_zero_point = output_details[1]['quantization']
predicted_output_dequant = dequantize_output(predicted_output, output_scale, output_zero_point)

# Convert raw output scores to predicted class by taking argmax.
predicted_label = np.argmax(predicted_output_dequant, axis=1)
true_label = Y_test[sample_index]

print("\n--- Single Sample Inference (Int8 Quantized for AE) ---")
print(f"Sample index: {sample_index}")
print(f"True label: {true_label}")
print(f"Predicted label: {predicted_label[0]}")
print("Raw predicted scores (dequantized):", predicted_output_dequant)

# For a single sample, accuracy is 1 if the prediction matches the true label, else 0.
single_accuracy = 1 if predicted_label[0] == true_label else 0
print(f"Accuracy (single sample): {single_accuracy}")

# Compute additional metrics for the single sample (they will be either 0 or 1)
y_true_single = np.array([true_label])
y_pred_single = np.array([predicted_label[0]])

from sklearn.metrics import f1_score, precision_score, recall_score
f1 = f1_score(y_true_single, y_pred_single, average='macro')
precision = precision_score(y_true_single, y_pred_single, average='macro', zero_division=0)
recall = recall_score(y_true_single, y_pred_single, average='macro', zero_division=0)

print(f"F1 Score (single sample): {f1:.4f}")
print(f"Precision (single sample): {precision:.4f}")
print(f"Recall (single sample): {recall:.4f}")


Using output index 1 for classification predictions.

--- Single Sample Inference (Int8 Quantized for AE) ---
Sample index: 5
True label: 3
Predicted label: 3
Raw predicted scores (dequantized): [[0.         0.         0.         0.99609375 0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]]
Accuracy (single sample): 1
F1 Score (single sample): 1.0000
Precision (single sample): 1.0000
Recall (single sample): 1.0000


In [8]:
# ---------------------------
# 4. Benchmark Inference Time and Compute FPS (Int8 Quantized)
# ---------------------------
n_runs = 10000
inference_times = []

for i in range(n_runs):
    start_time = time.perf_counter()
    # Run inference using our run_inference_tflite function.
    _ = run_inference_tflite(sample_quantized)  # sample_quantized is the pre-quantized sample.
    end_time = time.perf_counter()
    
    # Convert elapsed time from seconds to milliseconds.
    elapsed_ms = (end_time - start_time) * 1000
    inference_times.append(elapsed_ms)
    
    if (i + 1) % 100 == 0:
        print(f"Run {i+1}: {elapsed_ms:.4f} ms")

mean_time = np.mean(inference_times)
std_time = np.std(inference_times)
fps_values = 1000.0 / np.array(inference_times)  # FPS = 1000 ms / inference time in ms.
mean_fps = np.mean(fps_values)
std_fps = np.std(fps_values)

print(f"\nAverage inference time over {n_runs} runs: {mean_time:.4f} ms (std: {std_time:.4f} ms)")
print(f"Average FPS over {n_runs} runs: {mean_fps:.4f} FPS (std: {std_fps:.4f} FPS)")

gc.collect()

Run 100: 0.0175 ms
Run 200: 0.0175 ms
Run 300: 0.0174 ms
Run 400: 0.0168 ms
Run 500: 0.0164 ms
Run 600: 0.0163 ms
Run 700: 0.0162 ms
Run 800: 0.0162 ms
Run 900: 0.0163 ms
Run 1000: 0.0162 ms
Run 1100: 0.0163 ms
Run 1200: 0.0163 ms
Run 1300: 0.0163 ms
Run 1400: 0.0163 ms
Run 1500: 0.0162 ms
Run 1600: 0.0165 ms
Run 1700: 0.0162 ms
Run 1800: 0.0164 ms
Run 1900: 0.0276 ms
Run 2000: 0.0162 ms
Run 2100: 0.0164 ms
Run 2200: 0.0163 ms
Run 2300: 0.0164 ms
Run 2400: 0.0163 ms
Run 2500: 0.0162 ms
Run 2600: 0.0163 ms
Run 2700: 0.0163 ms
Run 2800: 0.0163 ms
Run 2900: 0.0163 ms
Run 3000: 0.0165 ms
Run 3100: 0.0163 ms
Run 3200: 0.0166 ms
Run 3300: 0.0163 ms
Run 3400: 0.0163 ms
Run 3500: 0.0164 ms
Run 3600: 0.0163 ms
Run 3700: 0.0163 ms
Run 3800: 0.0163 ms
Run 3900: 0.0163 ms
Run 4000: 0.0163 ms
Run 4100: 0.0163 ms
Run 4200: 0.0163 ms
Run 4300: 0.0163 ms
Run 4400: 0.0163 ms
Run 4500: 0.0163 ms
Run 4600: 0.0162 ms
Run 4700: 0.0164 ms
Run 4800: 0.0163 ms
Run 4900: 0.0163 ms
Run 5000: 0.0163 ms
Run 5100:

22